In [1]:
import pandas as pd
import numpy as np
import climb_jor
import climber_class

# Route recommender

In this notebook will perform the route recommending based on the user features

### Create the user from climber class

In [2]:
climber_1 = climber_class.climber()
climber_2 = climber_class.climber()
climber_2.set_attributes(cluster=[-3,4,11,-1,30,-29,3,2,8])
display(climber_2.get_data())

Climber class initialized
Climber class initialized


,climber_id,name,grade,grade_range,country,crag,sector,height,sex
0,1,climber1,54,2,esp,montserrat,agulla del senglar,170,0


### Retrieve routes table

In [3]:
routes = pd.read_csv('../data/routes_rated.csv',low_memory=False, index_col=0)
routes.head(2)

,name_id,country,crag,sector,name,tall_recommend_sum,grade_mean,cluster,rating_tot
0,0,and,montserrat,prohibitivo,el mehir,-1,49.0,0,-0.026141
1,1,and,montserrat,prohibitivo,el pas de la discordia,0,49.0,0,-0.085161


### Cleaning the table

Objective
- We will recommend 3 routes per each zone (country, crag, sector)

The logical tree will be
- Make 3 different filters for: sector, crag, country
- Filter tall recommended (if height<-1 and tall or if height>1 and small) tall is mean+5 (>180), short is mean-5 (<170)
- Filter by grade range, default +-2 (e.g. for a 7b grade => range[7a+,7b+]), if not routes ask to make the range bigger
- Order by overall rating
- Filter preferred clusters (If not then show other results)
- Sex just adds an informative value

#### a) make zone tables

In [4]:
climber_2.get_data()

,climber_id,name,grade,grade_range,country,crag,sector,height,sex
0,1,climber1,54,2,esp,montserrat,agulla del senglar,170,0


In [5]:
routes_country = routes[routes.country == climber_2.get_data().country[0]]
routes_crag = routes[routes.crag == climber_2.get_data().crag[0]]
routes_sector = routes[routes.sector == climber_2.get_data().sector[0]]
print("Country routes: ", routes_country.shape[0])
print("Crag routes: ", routes_crag.shape[0])
print("Sector routes: ", routes_sector.shape[0])

Country routes:  15010
Crag routes:  814
Sector routes:  17


#### b) Filter tall recommend

In [6]:
climber_1.set_attributes(height=165)
climber_2.set_attributes(height=185)

Test with short

In [7]:
routes_country_tall = climb_jor.filter_tall(routes_df = routes_country, climber_usr = climber_1, show = True)
routes_crag_tall = climb_jor.filter_tall(routes_df = routes_crag, climber_usr = climber_1, show = True)
routes_sector_tall = climb_jor.filter_tall(routes_df = routes_sector, climber_usr = climber_1, show = True)

df routes:  12289
df routes:  663
df routes:  16


Test with tall

In [8]:
routes_country_tall = climb_jor.filter_tall(routes_df = routes_country, climber_usr = climber_2, show = True)
routes_crag_tall = climb_jor.filter_tall(routes_df = routes_crag, climber_usr = climber_2, show = True)
routes_sector_tall = climb_jor.filter_tall(routes_df = routes_sector, climber_usr = climber_2, show = True)

df routes:  12564
df routes:  714
df routes:  14


#### c) Filter grade range

In [9]:
routes_country_grade = climb_jor.filter_grade(routes_df = routes_country_tall, climber_usr = climber_2, show = True)
routes_crag_grade = climb_jor.filter_grade(routes_df = routes_crag_tall, climber_usr = climber_2, show = True)
routes_sector_grade = climb_jor.filter_grade(routes_df = routes_sector_tall, climber_usr = climber_2, show = True)

df routes:  1722
df routes:  116
df routes:  1


#### d) Order by overall rating

In [10]:
routes_country_rating = routes_country_grade.sort_values(by = 'rating_tot',ascending = False)
routes_crag_rating = routes_crag_grade.sort_values(by = 'rating_tot',ascending = False)
routes_sector_rating = routes_sector_grade.sort_values(by = 'rating_tot',ascending = False)

#### d) Filter clustering

In [11]:
routes_country_grade = climb_jor.filter_cluster(routes_df = routes_country_rating, climber_usr = climber_2, show = True)
routes_crag_grade = climb_jor.filter_cluster(routes_df = routes_crag_rating, climber_usr = climber_2, show = True)
routes_sector_grade = climb_jor.filter_cluster(routes_df = routes_sector_rating, climber_usr = climber_2, show = True)

df routes:  1722
df routes:  116
df routes:  1


In [12]:
climber_2.get_cluster_order()

[4, 2, 8, 1, 6, 7, 3, 0, 5]

#### e) Filter sex plus

In [13]:
# routes.sex_ratio.mean()

In [14]:
# routes_country_sex = climb_jor.sex_plus(routes_df = routes_country_grade, climber_usr = climber_2, show = True, threshold= routes.sex_ratio.mean())
# routes_crag_sex = climb_jor.sex_plus(routes_df = routes_crag_grade, climber_usr = climber_2, show = True, threshold= routes.sex_ratio.mean())
# routes_sector_sex = climb_jor.sex_plus(routes_df = routes_sector_grade, climber_usr = climber_2, show = True, threshold= routes.sex_ratio.mean())

#### f) 3 per zone

In [15]:
display(routes_country_grade.head(3))
display(routes_crag_grade.head(3))
display(routes_sector_grade.head(3))

,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
16247,16247,esp,ibiza,punta aubarca,dreams will be destroyed,55.666667,4,4.431579,0
21338,21338,esp,reguchillo,belit,sucinas line,53.777778,4,2.025564,0
10674,10674,esp,archidona,la cueva,danza agresiva,52.500000,4,0.984974,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
19554,19554,esp,montserrat,la calavera,batura,54.500000,4,0.154580,0
19202,19202,esp,montserrat,cova de larcada,a mar oberta,54.666667,4,0.002074,0
19222,19222,esp,montserrat,cova de larcada,queda usted despedido,53.000000,4,-0.074180,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
18969,18969,esp,montserrat,agulla del senglar,discordia l,53.574074,3,0.622013,0


## Create the function

In [16]:
def route_recommender(climber, show = False):
    '''
    This function returns the route recommendations for the user "climber"
    inputs:
    - climber: the user from the climber class
    - show: True for showing the log
    
    output:
    df with the recommendations       
    '''
    
    
    def recommender_filter(df, climber, show = True):
        routes = df.copy()
         # Climber Filters    
        routes_tall = climb_jor.filter_tall(routes_df = routes, climber_usr = climber, show = show)
        routes_grade = climb_jor.filter_grade(routes_df = routes_tall, climber_usr = climber, show = show)

        # Rating Order
        routes_rating = routes_grade.sort_values(by = 'rating_tot',ascending = False)

        # Clustering Order
        routes_grade = climb_jor.filter_cluster(routes_df = routes_rating, climber_usr = climber, show = show)

        return routes_grade
    
    
    routes = pd.read_csv('../data/routes_rated.csv',low_memory=False, index_col=0)
       
    # Region
    routes_country = routes[routes.country == climber.get_data().country[0]]
    routes_crag = routes[routes.crag == climber.get_data().crag[0]]
    routes_sector = routes[routes.sector == climber.get_data().sector[0]]
    
    # Recommen
    routes_country_rec = recommender_filter(routes_country, climber, show)
    routes_crag_rec = recommender_filter(routes_crag, climber, show)
    routes_sector_rec = recommender_filter(routes_sector, climber, show)
    
    display(routes_country_rec.head(3))
    display(routes_crag_rec.head(3))
    display(routes_sector_rec.head(3))
       

In [17]:
route_recommender(climber_2)

,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
16247,16247,esp,ibiza,punta aubarca,dreams will be destroyed,55.666667,4,4.431579,0
21338,21338,esp,reguchillo,belit,sucinas line,53.777778,4,2.025564,0
10674,10674,esp,archidona,la cueva,danza agresiva,52.500000,4,0.984974,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
19554,19554,esp,montserrat,la calavera,batura,54.500000,4,0.154580,0
19202,19202,esp,montserrat,cova de larcada,a mar oberta,54.666667,4,0.002074,0
19222,19222,esp,montserrat,cova de larcada,queda usted despedido,53.000000,4,-0.074180,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
18969,18969,esp,montserrat,agulla del senglar,discordia l,53.574074,3,0.622013,0


In [20]:
a,b,c = climber_2.route_recommender(show = True)

df routes:  12564
df routes:  1722
df routes:  1722
df routes:  714
df routes:  116
df routes:  116
df routes:  14
df routes:  1
df routes:  1


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
16247,16247,esp,ibiza,punta aubarca,dreams will be destroyed,55.666667,4,4.431579,0
21338,21338,esp,reguchillo,belit,sucinas line,53.777778,4,2.025564,0
10674,10674,esp,archidona,la cueva,danza agresiva,52.500000,4,0.984974,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
19554,19554,esp,montserrat,la calavera,batura,54.500000,4,0.154580,0
19202,19202,esp,montserrat,cova de larcada,a mar oberta,54.666667,4,0.002074,0
19222,19222,esp,montserrat,cova de larcada,queda usted despedido,53.000000,4,-0.074180,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
18969,18969,esp,montserrat,agulla del senglar,discordia l,53.574074,3,0.622013,0


In [21]:
cl = climber_class.climber()

a,b,c = cl.route_recommender(show = True)

Climber class initialized
df routes:  15010
df routes:  2041
df routes:  2041
df routes:  814
df routes:  129
df routes:  129
df routes:  17
df routes:  1
df routes:  1


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
23685,23685,esp,sorrueda,salon,el pizco del zapa,53.0,0,2.535027,0
18851,18851,esp,montgrony,la vena,el cap de la vena,55.0,0,2.415084,0
12150,12150,esp,can ortigues,carretera,regina,53.0,0,1.884547,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
19400,19400,esp,montserrat,el vermell,la tele del menguele,52.666667,0,1.261773,0
19268,19268,esp,montserrat,el vermell,amb el temps,54.333333,0,0.989732,0
19608,19608,esp,montserrat,odio africano,amb el temps creixem,54.000000,0,0.984974,0


,name_id,country,crag,sector,name,grade_mean,cluster,rating_tot,height_plus
18969,18969,esp,montserrat,agulla del senglar,discordia l,53.574074,3,0.622013,0
